# 变量：创建、初始化、保存和加载

## 创建  
当创建一个变量时，你将一个**张量Tensor**作为初始值闯入构造函数$tf.Variable()$中。Tensorflow提供了一系列操作符来初始化张量，初始值是常量或随机值。  
**注意**：所有这些操作符都需要你指定张量的shape。这个shape自动变成创建的变量Variable的shape。变量的shape一般是固定的，但Tensorflow提供了高级的机制来重新调整其行列数。

In [1]:
import tensorflow as tf

C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [3]:
# 创建两个变量，并没有完成初始化，类似只是做了声明
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35), name='weights')
biases = tf.Variable(tf.zeros([200]), name='biases')

In [4]:
print(weights.shape)
print(biases.shape)

(784, 200)
(200,)


## 初始化  
变量的初始化必须在模型的其他操作运行前先明确地完成。最简单的方法就是添加一个给所有变量初始化的操作，并在使用模型之前首先运行那个操作。  
使用tf.initialize_all_variables()，现在是tf.global_variables_initializer()添加一个操作对变量做初始化。记得在完全构建号模型并加载之后再运行该操作。

In [5]:
# 增加初始化变量的操作
init_op = tf.global_variables_initializer()

## 由另一个变量初始化  
有时候会需要用另一个变量的初始化值给当前变量初始化。由于tf.global_variables_initializer()是并行地初始化所有变量，所以再有这种需求的情况下需要小心。  
  
用其他变量的值初始化一个新的变量时，使用其他变量的initialized_value()属性。  
可以直接把已初始化的值作为新变量的初始化值，或者把它当作tensor计算得到一个值赋予新变量。

In [6]:
# 用一个和weights同样的初始化值创建新变量
w2 = tf.Variable(weights.initialized_value(), name='w2')
w_twice = tf.Variable(weights.initialized_value() * 0.2, name='w_twice')

w2.initialized_value()

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


<tf.Tensor 'cond_2/Merge:0' shape=(784, 200) dtype=float32>

## 保存和加载  
最简单的保存和加载模型的方法是使用tf.train.Saver对象。构造器给graph的所有变量，或是定义在列表里的变量，添加**save**和**restore**的操作。Saver对象提供了方法来运行这些操作，定义检查点文件的读写路径。

### 检查点文件   
变量存储在二进制文件里，主要包含从变量名到tensor值的映射关系。  
当你创建一个Saver对象时，你可以选择性地为检查点文件中的变量挑选变量名。默认情况下为每个变量Varaible.name属性的值。

### 保存变量

In [7]:
# 添加一个Saver对象存储所有变量
saver = tf.train.Saver()

### 恢复变量  
用同一个Saver对象来恢复变量。注意，当你从文件中恢复变量时，不需要事先对它们做初始化。